# Housing sales Price and Venues Data Analysis in Jakarta

## A. Introduction

### A.1 Description and Discussion Background

Jakarta has most density in Indonesia with 9.6 million people live. It has 661,5 SqKm with 14.464 people per square kilometer. The city is divided to 5 borough East, West, South,North and Center Jakarta. As an Indonesian, I use my capital city in my project. As a capital city and a business city, Jakarta has high price real estates. If people want to live in Jakarta, it should cost around Rp. 2.000.0000 - Rp. 5.000.000 average per month to rent an apartment or a boarding house, and around Rp. 500.000.000 - Rp. 1.000.000.000 for an apartment, of course it depends on the location. When we start to think to live in Jakarta, we may want to choose the best location for us with vary venues like cafe, hospital, etc and low cost absolutely. However it is difficult to obtain the information to determine which real estate we should buy. Will the house really worth it or not.

When we consider all the parameter, we can create a information chart where the real estate is placed and each district is clutered according to the venues.

### A.2 Data Description

This is the data I will use:

- https://ekonomi.bisnis.com/read/20171110/48/708065/berikut-pergerakana-harga-properti-di-jakarta Average house price in jakarta
- Foursquare API to show venues around the borough
- Jakarta geojson from https://gist.github.com/heffergm/d261f9cf3b334b8e523ee7fe0b3ef583?short_path=8e72f60

## B. Methodology

I used github to store my database. My main data include Borough, average pricing house in Jakarta based on ekonomi.bisnis.com, latitude and longitude. I used python for my programming language cause it has a tons of library that can help me to analyze. I used folium to visualize geographic detail of Jakarta.



1. Creating dataframe from news data

In [28]:
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import folium
import requests
from shapely.geometry import Point, Polygon, LineString
import json

In [2]:
datas = [['East Jakarta',7900,-6.225,106.900],['West Jakarta',13200,-6.167,106.763],['North Jakarta',17100,-6.155,106.892],['South Jakarta',17900,-6.261,106.810],['Center Jakarta',18700,-6.2088,106.845]]
df = pd.DataFrame(datas)
df.columns = ['Borugh','Avg Price','Latitude','Longitude']
df

,Borugh,Avg Price,Latitude,Longitude
0,East Jakarta,7900,6.2250,106.900
1,West Jakarta,13200,6.1670,106.763
2,North Jakarta,17100,6.1550,106.892
3,South Jakarta,17900,6.2610,106.810
4,Center Jakarta,18700,6.2088,106.845


2. Creating Jakarta map separated by sub-district

In [5]:
jakarta_lat = -6.2088
jakarta_long = 106.8456
state_geo = 'jakarta_by_kelurahan_geo.geojson'

m = folium.Map(location=[jakarta_lat,jakarta_long], zoom_start=12)

folium.Choropleth(
    geo_data=state_geo,
    name='choropleth',
    fill_color='BuPu',
    fill_opacity=0.3,
    line_opacity=0.8
).add_to(m)

folium.LayerControl().add_to(m)

m

3. Marking every center of sub-district

In [49]:


with open('jakarta_by_kelurahan_geo.geojson') as f:
    data = json.load(f)

# data['features']
for feature in data['features']:
#     print(feature['geometry']['coordinates'][0][0])
    p = Polygon(feature['geometry']['coordinates'][0][0])
    point = p.centroid
    
    folium.CircleMarker(
        [point.y, point.x],
        radius=5,
        color='blue',
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(m)

In [50]:
m

4 Creating function for generate venues arround sub-district

In [ ]:
CLIENT_ID = 'VF1SBTXRNXKEDALWBRLRE2NT2K0Z3WVTUD3TGPV2DA5HOB3B' # your Foursquare ID
CLIENT_SECRET = 'WQI03FWI4MIEQVY0PRJYDAUOZJA1P0COTYGYSMFVQ2VXLAEI' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 500
categoryId='4d4b7105d754a06374d81259' #category id for food category


# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']

    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

def generate_venue(lat,lng):
    ll = '{},{}'.format(lat,lng)
    sortByPopularity = '1'
    print('Your credentails:')
    print('CLIENT_ID: ' + CLIENT_ID)
    print('CLIENT_SECRET:' + CLIENT_SECRET)
    search_query = ''
    radius = 500
    print(search_query + ' .... OK!')
    url = 'https://api.foursquare.com/v2/venues/search?ll={}&sortByPopularity={}&client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ll,sortByPopularity,CLIENT_ID, CLIENT_SECRET, jakarta_lat, jakarta_long, VERSION, search_query, radius, LIMIT)

    
    results = requests.get(url).json()
    
    # assign relevant part of JSON to venues
    venues = results['response']['venues']

    # tranform venues into a dataframe
    dataframe = json_normalize(venues)
    
    
    # keep only columns that include venue name, and anything that is associated with location
    filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
    dataframe_filtered = dataframe.loc[:, filtered_columns]

    # filter the category for each row
    dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

    # clean column names by keeping only last term
    dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

    for lati, long, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lati, long],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(m)

In [165]:
CLIENT_ID = 'VF1SBTXRNXKEDALWBRLRE2NT2K0Z3WVTUD3TGPV2DA5HOB3B' # your Foursquare ID
CLIENT_SECRET = 'WQI03FWI4MIEQVY0PRJYDAUOZJA1P0COTYGYSMFVQ2VXLAEI' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 500
categoryId='4d4b7105d754a06374d81259' #category id for food category
near='Jembatan + lima'
ll = '-6.2250,106.900'
sortByPopularity = '1'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

search_query = ''
radius = 500
print(search_query + ' .... OK!')

url = 'https://api.foursquare.com/v2/venues/search?categoryId={}&ll={}&sortByPopularity={}&client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(categoryId,ll,sortByPopularity,CLIENT_ID, CLIENT_SECRET, jakarta_lat, jakarta_long, VERSION, search_query, radius, LIMIT)
url

Your credentails:
CLIENT_ID: VF1SBTXRNXKEDALWBRLRE2NT2K0Z3WVTUD3TGPV2DA5HOB3B
CLIENT_SECRET:WQI03FWI4MIEQVY0PRJYDAUOZJA1P0COTYGYSMFVQ2VXLAEI
 .... OK!


'https://api.foursquare.com/v2/venues/search?categoryId=4d4b7105d754a06374d81259&ll=-6.2250,106.900&sortByPopularity=1&client_id=VF1SBTXRNXKEDALWBRLRE2NT2K0Z3WVTUD3TGPV2DA5HOB3B&client_secret=WQI03FWI4MIEQVY0PRJYDAUOZJA1P0COTYGYSMFVQ2VXLAEI&ll=-6.2088,106.8456&v=20180604&query=&radius=500&limit=2150'

In [177]:
venue_id = '4fd81ed3e4b0859a9e5f25bc' # ID of Harry's Italian Pizza Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

results = requests.get(url).json()

try:
    print(results)
except:
    print('This venue has not been rated yet.')
    
results['response']['venue']['tips']['count']


{'meta': {'code': 200, 'requestId': '5fc4aa963689252594c154ed'}, 'response': {'venue': {'id': '4fd81ed3e4b0859a9e5f25bc', 'name': 'A&W mal ambassador', 'contact': {}, 'location': {'address': 'Jlm Dr Satrio', 'lat': -6.228779841326133, 'lng': 106.89899773044193, 'labeledLatLngs': [{'label': 'display', 'lat': -6.228779841326133, 'lng': 106.89899773044193}], 'cc': 'ID', 'city': 'Jakarta', 'state': 'Jakarta', 'country': 'Indonesia', 'formattedAddress': ['Jlm Dr Satrio', 'Jakarta', 'Jakarta', 'Indonesia']}, 'canonicalUrl': 'https://foursquare.com/v/aw-mal-ambassador/4fd81ed3e4b0859a9e5f25bc', 'categories': [{'id': '4bf58dd8d48988d14e941735', 'name': 'American Restaurant', 'pluralName': 'American Restaurants', 'shortName': 'American', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_', 'suffix': '.png'}, 'primary': True}], 'verified': False, 'stats': {'tipCount': 0}, 'price': {'tier': 3, 'message': 'Expensive', 'currency': '$'}, 'likes': {'count': 0, 'groups': []}, 'di

0

In [114]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

<ipython-input-114-5acf500bf9ad>:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  dataframe = json_normalize(venues)


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.postalCode
0,4d7d3e0a18a1ba7a8db24915,Bakso Selera,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1606721481,False,Jl. Balai Rakyat,-6.224548,106.899478,"[{'label': 'display', 'lat': -6.22454761785260...",76,ID,Jakarta,Jakarta,Indonesia,"[Jl. Balai Rakyat, Jakarta, Jakarta, Indonesia]",NaN,NaN
1,4c1b612db4e62d7fb938d993,Soto sate H.Nawi klender,"[{'id': '4bf58dd8d48988d142941735', 'name': 'A...",v-1606721481,False,Jl. Pahlawan revolusi,-6.223527,106.899269,"[{'label': 'display', 'lat': -6.22352661017134...",182,ID,Jakarta,Jakarta,Indonesia,"[Jl. Pahlawan revolusi, Jakarta timur, Jakarta...",NaN,NaN
2,55e3dfe5498ede7fd70e33f6,Sate Ayam Blora Yudi,"[{'id': '52960eda3cf9994f4e043ac7', 'name': 'J...",v-1606721481,False,Jl. Pahlawan Revolusi,-6.224977,106.899378,"[{'label': 'display', 'lat': -6.22497652548498...",68,ID,Jakarta,Jakarta,Indonesia,"[Jl. Pahlawan Revolusi (Klender), Jakarta, Jak...",Klender,NaN
3,54394326498eaf92fe528ccf,Mie Aceh Pandrah,"[{'id': '52960eda3cf9994f4e043ac9', 'name': 'A...",v-1606721481,False,Jalan Sukarjo Wiryopranoto No. 4C,-6.229805,106.897615,"[{'label': 'display', 'lat': -6.2298054, 'lng'...",596,ID,Sawah Besar,Jakarta,Indonesia,"[Jalan Sukarjo Wiryopranoto No. 4C, Sawah Besa...",NaN,NaN
4,5e887e11a305830007ef56a3,Wedang Jahe Tempe Mendoan,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",v-1606721481,False,Jl Pahlawan Revolusi,-6.223689,106.899107,"[{'label': 'display', 'lat': -6.22368857988357...",176,ID,Kecamatan Duren Sawit,Jakarta,Indonesia,[Jl Pahlawan Revolusi (Sebrang Apartemen Casab...,Sebrang Apartemen Casablanca East Residences,13430


In [115]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,crossStreet,postalCode,id
0,Bakso Selera,Restaurant,Jl. Balai Rakyat,-6.224548,106.899478,"[{'label': 'display', 'lat': -6.22454761785260...",76,ID,Jakarta,Jakarta,Indonesia,"[Jl. Balai Rakyat, Jakarta, Jakarta, Indonesia]",NaN,NaN,4d7d3e0a18a1ba7a8db24915
1,Soto sate H.Nawi klender,Asian Restaurant,Jl. Pahlawan revolusi,-6.223527,106.899269,"[{'label': 'display', 'lat': -6.22352661017134...",182,ID,Jakarta,Jakarta,Indonesia,"[Jl. Pahlawan revolusi, Jakarta timur, Jakarta...",NaN,NaN,4c1b612db4e62d7fb938d993
2,Sate Ayam Blora Yudi,Javanese Restaurant,Jl. Pahlawan Revolusi,-6.224977,106.899378,"[{'label': 'display', 'lat': -6.22497652548498...",68,ID,Jakarta,Jakarta,Indonesia,"[Jl. Pahlawan Revolusi (Klender), Jakarta, Jak...",Klender,NaN,55e3dfe5498ede7fd70e33f6
3,Mie Aceh Pandrah,Acehnese Restaurant,Jalan Sukarjo Wiryopranoto No. 4C,-6.229805,106.897615,"[{'label': 'display', 'lat': -6.2298054, 'lng'...",596,ID,Sawah Besar,Jakarta,Indonesia,"[Jalan Sukarjo Wiryopranoto No. 4C, Sawah Besa...",NaN,NaN,54394326498eaf92fe528ccf
4,Wedang Jahe Tempe Mendoan,Food Truck,Jl Pahlawan Revolusi,-6.223689,106.899107,"[{'label': 'display', 'lat': -6.22368857988357...",176,ID,Kecamatan Duren Sawit,Jakarta,Indonesia,[Jl Pahlawan Revolusi (Sebrang Apartemen Casab...,Sebrang Apartemen Casablanca East Residences,13430,5e887e11a305830007ef56a3
5,Sushi bento,Sushi Restaurant,NaN,-6.222582,106.897333,"[{'label': 'display', 'lat': -6.2225821, 'lng'...",399,ID,NaN,NaN,Indonesia,[Indonesia],NaN,NaN,4fec2aeee4b0044db3e3f83e
6,Tukang Pop Ice,Café,"Jalan Pahlawan Revolusi, Jakarta",-6.228198,106.898788,"[{'label': 'display', 'lat': -6.22819805145263...",380,ID,NaN,Jakarta,Indonesia,"[Jalan Pahlawan Revolusi, Jakarta, Jakarta, In...",NaN,NaN,4f669d91e4b0777dfe618385
7,A&W mal ambassador,American Restaurant,Jlm Dr Satrio,-6.228780,106.898998,"[{'label': 'display', 'lat': -6.22877984132613...",435,ID,Jakarta,Jakarta,Indonesia,"[Jlm Dr Satrio, Jakarta, Jakarta, Indonesia]",NaN,NaN,4fd81ed3e4b0859a9e5f25bc
8,Dunkin',Donut Shop,NaN,-6.225506,106.904810,"[{'label': 'display', 'lat': -6.2255059, 'lng'...",535,ID,NaN,NaN,Indonesia,[Indonesia],NaN,NaN,548b2186498e96b6cc6487fb
9,Bakso Moncrotz,Indonesian Restaurant,Jl.Tebet,-6.223230,106.899320,"[{'label': 'display', 'lat': -6.22323, 'lng': ...",210,ID,NaN,NaN,Indonesia,"[Jl.Tebet, Indonesia]",NaN,NaN,4ec73bb5f79041351c77ab67


In [116]:
dataframe_filtered.name

0                                         Bakso Selera
1                             Soto sate H.Nawi klender
2                                 Sate Ayam Blora Yudi
3                                     Mie Aceh Pandrah
4                            Wedang Jahe Tempe Mendoan
5                                          Sushi bento
6                                       Tukang Pop Ice
7                                   A&W mal ambassador
8                                              Dunkin'
9                                       Bakso Moncrotz
10                                     Es Cendol Duren
11                                                 KFC
12                               Pecel lele pecel ayam
13                                        Caffe Minang
14                         Jhonny's CAFE (JOCKER SPOT)
15                   Dunkin' donuts Mall Citra Klender
16                   Pondok Sate Tegal Ibu Hj. Jamilah
17                                           Czech 001
18        

In [117]:
venues_map = folium.Map(location=[jakarta_lat, jakarta_long], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [jakarta_lat, jakarta_long],
    radius=10,
    color='red',
    popup='',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(m)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(m)

# display map
m

In [119]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.
